In [9]:
import findspark
findspark.init()

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import os

spark = SparkSession.builder.master("local").appName("Accumulators").getOrCreate()
sc = spark.sparkContext

In [11]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
spark.conf.set("spark.sql.shuffle.partitions",3)
spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.adaptive.enabled","false")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","false")
spark.conf.set("spark.sql.execution.arrow.enabled",True)
spark.conf.set("spark.sql.execution.arrow.fallback.enabled",True)

"""
Alternative to DF count operation
1.Read the input
2.Apply filter operation
3.groupBy aggregate
4.show grouped data

get count of data without using count operation

accumulator are counter or shared variable in spark
accumulator has four call, accum_parameter,add,value
accumulator should be applied on action, applying on transformation would misled the count
"""

#WithoutInferSchema
#headerTrue will read first row and assign column names but type is String for all
#spark job created to read first column
filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"
df = spark.read.option("header",True) \
            .option("delimiter","|") \
            .option("inferSchema",True) \
            .csv(filepath + "IntUspopulation.csv")

In [13]:
df.count()

156

In [16]:
#Filter data
df1 = df.filter("State_code <> 'NY'").groupBy("city").sum("2019_estimate")

In [17]:
df1.count()

147

In [24]:
#accumulator
dfcounter = spark.sparkContext.accumulator(0)
filtercounter = spark.sparkContext.accumulator(0)

In [27]:
df.foreach(lambda x: dfcounter.add(1))
df1.foreach(lambda x: filtercounter.add(1))

In [28]:
print(dfcounter.value)

156


In [30]:
print(filtercounter.value)

147
